Notatnik do tworzenia skryptu Python urachamianego przez Glue Joba 

In [43]:
%region eu-west-1
%profile superuser-test    
%iam_role arn:aws:iam::866525145675:role/LSM-DP-Genius-Rec-Glue-Service-Role-test
%glue_version 3.0
%number_of_workers 2
%worker_type G.1X
%idle_timeout 60
%session_id_prefix grsi-dp
%session_type ETL
%extra_jars s3://lsm-dp-utility-eu-test/jre/sqljdbc41.jar
#%security_config grsi-dp-security-config-test
%connections GeniusAnalyticalRep-RDS-test
%additional_python_modules s3://lsm-dp-utility-eu-test/python-library/lsm-dp-library.zip


You are already connected to a glueetl session grsi-dp-7014c44a-3f75-4305-bd8f-4abc4c9e0da0.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous region: eu-west-1
Setting new region to: eu-west-1
Region is set to: eu-west-1


You are already connected to a glueetl session grsi-dp-7014c44a-3f75-4305-bd8f-4abc4c9e0da0.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous profile: superuser-test
Setting new profile to: superuser-test


You are already connected to a glueetl session grsi-dp-7014c44a-3f75-4305-bd8f-4abc4c9e0da0.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


...

Extra jars to be included:
s3://lsm-dp-utility-eu-test/jre/sqljdbc41.jar
s3://lsm-dp-utility-eu-test/jre/sqljdbc41.jar


You are already connected to a glueetl session grsi-dp-7014c44a-3f75-4305-bd8f-4abc4c9e0da0.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Connections to be included:
GeniusAnalyticalRep-RDS-test


You are already connected to a glueetl session grsi-dp-7014c44a-3f75-4305-bd8f-4abc4c9e0da0.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Additional python modules to be included:
s3://lsm-dp-utility-eu-test/python-library/lsm-dp-library.zip


In [20]:
%%tags 
{
    "lm_app":"aws-dp", 
    "lm_app_env":"test", 
    "lm_sbu":"GS", 
    "lm_troux_uid":"B3439EB1-509F-4193-B1B3-2023D75CBD29", 
    "Name":"grsi-dp-int-glue-sesion-test", 
    "deployment_guid":"2f72a84f-6093-4813-8781-77ca784427b8",
    "organization_guid":"2326c977-2f8d-4d97-8f13-a7ca2fd5d5db"
}

Tag {'lm_app': 'aws-dp', 'lm_app_env': 'test', 'lm_sbu': 'GS', 'lm_troux_uid': 'B3439EB1-509F-4193-B1B3-2023D75CBD29', 'Name': 'grsi-dp-int-glue-sesion-test', 'deployment_guid': '2f72a84f-6093-4813-8781-77ca784427b8', 'organization_guid': '2326c977-2f8d-4d97-8f13-a7ca2fd5d5db'} added


In [9]:
import boto3
import json
import sys
import os
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.dynamicframe import DynamicFrame
from awsglue.job import Job
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.sql.functions import *

import datetime


In [ ]:
# %pip install awswrangler
# %pip install psycopg2

%help

In [10]:
import awswrangler


ModuleNotFoundError: No module named 'awswrangler'


In [ ]:




redshift = {
    "host" : "grsi-dp-redshift-eu-test.c7rb8efwhoho.eu-west-1.redshift.amazonaws.com",
    "port" : "5439",
    "database" : "grsi_dp_eu_test"
}


redshift_url = "jdbc:redshift://{host}:{port}/{database}".format(
    host = redshift["host"],
    port = redshift["port"],
    database = redshift["database"],
)


#Setting connection
conn_options={
    "host" : redshift["host"],
    "port" : redshift["port"],
    "database" : redshift["database"],
    "url": redshift_url,
    "dbtable": "reconciliation.dbt_test_results",
    "redshiftTmpDir": "s3://grsi-dp-ingestion-eu-test/temp",
    "aws_iam_role": "arn:aws:iam::866525145675:role/grsi-dp-ingestion-lambda-role-eu-test",
    "user": "v-token-pa-break-gl-eys6symaiaf7aloi6liq-1733730019",
    "password": "xz-XysnZ1BaoX2hiwh0M",
    "driver": "com.amazon.redshift.jdbc.Driver"
    }

awr_rs_connection = {
     "host" : conn_options["host"], 
     "username" : conn_options["user"], 
     "password" : conn_options["password"], 
      "engine" : "redshift", 
      "port" : conn_options["port"], 
      "dbname" : conn_options["database"], 
}

print (awr_rs_connection)

In [ ]:
spark = SparkSession.builder \
    .appName("Redshift Connection with PySpark") \
    .getOrCreate()

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark_s = glueContext.spark_session

In [ ]:
print(spark_s.sparkContext)
print(spark_s.sparkContext.appName)

In [ ]:
df = spark_s.read \
    .format("jdbc") \
    .option("url", conn_options["url"]) \
    .option("dbtable", conn_options["dbtable"]) \
    .option("user", conn_options["user"]) \
    .option("password", conn_options["password"]) \
    .option("driver", conn_options["driver"]) \
    .load()

In [ ]:
query = "(select * from reconciliation.dbt_test_results) as tmp"

print(query)

df2 = spark.read.jdbc(redshift_url, query, properties=conn_options)

print (df2.show())

In [ ]:
# Wybrane kolumny

spark_result_tables = SparkSession.builder \
    .appName("Result Tables") \
    .getOrCreate()


query_distinct = "(select distinct database_name, schema_name,detail_table_name, cast('0' as INT) done from reconciliation.dbt_test_results) as temp2"

print(query_distinct)

df_result_data = spark_result_tables.read.jdbc(redshift_url, query_distinct, properties=conn_options)

print (df_result_data.show())




In [ ]:

df_tables = df_result_data.select(df_result_data.detail_table_name)
df_tables.show(truncate=False)



In [ ]:

for row in df_tables.collect():
    extract_query = 'select * from {}'.format(row["detail_table_name"])
    print (extract_query)


In [ ]:
s3_bucket_path = 'arn:aws:s3:::grsi-dp-source-data-test/unload_recon/'
iam_role = 'arn:aws:iam::866525145675:role/LSM-DP-Genius-Rec-Glue-Service-Role-test'
sql_query = 'select * from grsi_dp_eu_test.staging_test.coverage_sums_whole_limit'

unload_command = "UNLOAD ('{}') TO '{}' IAM_ROLE '{}' PARQUET;" \
        .format(sql_query,
                s3_bucket_path + str(datetime.datetime.now()) + ".parquet",
                iam_role,
                )

print(unload_command)

In [ ]:
client = boto3.client('redshift-data')
response = client.execute_statement(
    ClusterIdentifier='grsi-dp-redshift-eu-test',
    Database=redshift_url['database'],
    DbUser=conn_options['user'],
    Sql=unload_command
)


con = awswrangler.redshift.connect("MY_GLUE_CONNECTION")
df = wr.redshift.unload(
    sql="SELECT * FROM public.mytable",
    path="s3://bucket/extracted_parquet_files/",
    con=con,
    iam_role="arn:aws:iam::XXX:role/XXX"
)
con.close()


# redshift_url = "jdbc:redshift://{host}:{port}/{database}".format(
#     host="grsi-dp-redshift-eu-test.c7rb8efwhoho.eu-west-1.redshift.amazonaws.com",
#     port="5439",
#     database="grsi_dp_eu_test"
# )

# #Setting connection
# conn_options={
#     "url": redshift_url,
#     "dbtable": "reconciliation.dbt_test_results",
#     "redshiftTmpDir": "s3://grsi-dp-ingestion-eu-test/temp",
#     "aws_iam_role": "arn:aws:iam::866525145675:role/grsi-dp-ingestion-lambda-role-eu-test",
#     "user": "v-token-pa-break-gl-eys6symaiaf7aloi6liq-1733730019",
#     "password": "xz-XysnZ1BaoX2hiwh0M",
#     "driver": "com.amazon.redshift.jdbc.Driver"
#     }